In [ ]:
import panel as pn
import param
import os

In [ ]:
PORTATIL_DEV = False

if PORTATIL_DEV:
    CSV_PATH = "C:/Users/david/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
else:
    CSV_PATH = "C:/Users/david/OneDrive/MrRobot/IITV/4/TFG/data-Vito-PC/"

data_directory = os.listdir(CSV_PATH)

class TFG(param.Parameterized):
    database  = param.Selector(objects=data_directory)
    algorithm  = param.Selector(objects=['Genetic Algorithm', 
                                         'Particle Swarm Optimization', 
                                         'Coral Reef Optimization'])
    generations = param.Integer(default=10, bounds=(1, 50))
    population_size = param.Number(default=10, bounds=(1, 200), step=1)
    hidden_neurons = param.Integer(default=1, bounds=(0, 1000), step=50)
    complexity = param.Selector(objects=[1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3], default=1)

    #files = param.MultiFileSelector(path='../../data-Vito-PC/*.csv?', precedence=0.5)
    # if selected algorithm is GA, then implement a parameter for the number of generations



In [ ]:
# DEFAULT VALUES
default_value = 50
x = 50

# PARAMS
basic_params = TFG()

# GA PARAMS
crossover_probability_slider = pn.widgets.FloatSlider(name='Crossover Probability', start=0, end=1, value=0.5)
mutation_probability_slider = pn.widgets.FloatSlider(name='Mutation Probability', start=0, end=1, value=0.5)

# PSO PARAMS
inertia_weight_slider = pn.widgets.FloatSlider(name='Inertia Weight', start=0, end=1, value=0.5)
cognitive_component_slider = pn.widgets.FloatSlider(name='Cognitive Component', start=0, end=1, value=0.5)
social_component_slider = pn.widgets.FloatSlider(name='Social Component', start=0, end=1, value=0.5)

# CRO PARAMS
reef_size_slider = pn.widgets.IntSlider(name='Reef Size', start=0, end=100, value=50)
occupied_corals_slider = pn.widgets.IntSlider(name='Occupied Colonies', start=0, end=reef_size_slider.value, value=50)

# Sliders
value_slider = pn.widgets.FloatSlider(name='Iterations', start=0.0, end=100, value=default_value)


# Additional Widgets
label_widget = pn.widgets.StaticText(name='Label', value=x, width=200, height=50)
configuration = pn.widgets.StaticText(name='Configuration', value='', width=500, height=50)

reset_params_button = pn.widgets.Button(name='Reset Parameters', width=50)
run_button = pn.widgets.Button(name='\u25b6 Begin algorithm', width=50)
next_generation_button = pn.widgets.Button(name='Next Generation', width=50)

# Events
def reset_event(event):
    global default_value
    global x
    value_slider.value = default_value
    label_widget.value = default_value
    x = default_value

def display_configuration(event):
    configuration.value = f"""\nSelected database: {basic_params.database},
                                and selected algorithm: {basic_params.algorithm},
                                with {basic_params.generations} generations,
                                {basic_params.population_size} population size,
                                {basic_params.hidden_neurons} hidden neurons,
                                and complexity {basic_params.complexity}
                    """

def next_gen_event(event):
    global x
    x += 1
    label_widget.value = x

# Commands
reset_params_button.on_click(reset_event)
run_button.on_click(display_configuration)
next_generation_button.on_click(next_gen_event)

In [ ]:
# Set responsive panel
pn.extension(sizing_mode='stretch_width')

instructions = pn.pane.Markdown('''
# Bio-Inspired Optimization Algorithms in Extreme Learning Machine
## Instructions: 
1. **Click on the plot to place the target.** 
2. Click '\u25b6 Begin Improving' button to begin improving for the time on the Time Evolving slider. 
3. Experiment with the sliders 
''')

ga_column = pn.Column(crossover_probability_slider, mutation_probability_slider)
pso_column = pn.Column(inertia_weight_slider, cognitive_component_slider, social_component_slider)
cro_column = pn.Column(reef_size_slider, occupied_corals_slider)

dashboard = pn.Column(instructions, 
                      pn.Row(pn.Column(label_widget, configuration), pn.Column(basic_params,value_slider, run_button, next_generation_button, reset_params_button))
                     )

dashboard.background = '#f0f0f0'

def update_dashboard(event):
        if basic_params.algorithm == 'Genetic Algorithm':
                dashboard[1][1][1] = ga_column
        elif basic_params.algorithm == 'Particle Swarm Optimization':
                dashboard[1][1][1] = pso_column
        elif basic_params.algorithm == 'Coral Reef Optimization':
                dashboard[1][1][1] = cro_column

basic_params.param.watch(update_dashboard, 'algorithm')

# Open the dashboard in a local server 
dashboard.show(port=5006)

